In [1]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
! pip install openai
! pip install tiktoken

  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/cb/bb/c51fa42d85f431b3b3ec4c35a13a8bb99cafc0671918139a48767421d354/pinecone_client-3.2.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/215.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/215.9 kB ? eta -:--:--
   --- ----------------------------------- 20.5/215.9 kB 217.9 kB/s eta 0:00:01
   ------- ------------------------------- 41.0/215.9 kB 245.8 kB/s eta 0:00:01
   --------- ----------------------------- 51.2/215.9 kB 238.1 kB/s eta 0:00:01
   -------------- ------------------------ 81.9/215.9 kB 327.3 kB/s eta 0:00:01
   ----------------------- -------------- 133.1/215.9 kB 462.0 kB/s eta 0:00:01
   -------------------------------- ----- 184.3/215.9 kB 556.2 kB/s eta 0:00:01
   -------------------------------------- 215.9/215.9 kB 571.3 kB/s eta 0:00:00


In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [3]:
!mkdir pdfs

In [6]:
loader=PyPDFDirectoryLoader("pdfs")

In [8]:
data=loader.load()

In [10]:
data[0]

Document(page_content='YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known\nreal-time object detectors with 30 FPS or higher on GPU\nV100. YOLOv7-E6 object detector (56 FPS V100, 55.9%\nAP) outperforms both transformer-based detector SWIN-\nL Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by\n509% in speed and 2% in accuracy, and convolutional-\nbased detector ConvNeXt-XL Cascade-Mask R-CNN (8.6\nFPS A100, 55.2% AP) by 551% in speed and 0.7% AP\nin accuracy, as well as YOLOv7 outperforms: YOLOR,\nYOLOX, Scaled-YOLOv4, YOLOv5, DETR, Deformable\nDETR, DINO-5scale-R50, ViT-Adapter-B and 

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [12]:
text_chunks = text_splitter.split_documents(data)

In [13]:
text_chunks

[Document(page_content='YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known', metadata={'source': 'pdfs/yolov7_compressed.pdf', 'page': 0}),
 Document(page_content='real-time object detectors with 30 FPS or higher on GPU\nV100. YOLOv7-E6 object detector (56 FPS V100, 55.9%\nAP) outperforms both transformer-based detector SWIN-\nL Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by\n509% in speed and 2% in accuracy, and convolutional-\nbased detector ConvNeXt-XL Cascade-Mask R-CNN (8.6\nFPS A100, 55.2% AP) by 551% in speed and 0.7% AP\nin accuracy, as well as YOLOv7 outperforms: YOLOR,\n

In [19]:
len(text_chunks)

152

In [16]:
print(text_chunks[0].page_content)

YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object
detectors
Chien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1
1Institute of Information Science, Academia Sinica, Taiwan
kinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw
Abstract
YOLOv7 surpasses all known object detectors in both
speed and accuracy in the range from 5 FPS to 160 FPS
and has the highest accuracy 56.8% AP among all known


In [17]:
print(text_chunks[1].page_content)

real-time object detectors with 30 FPS or higher on GPU
V100. YOLOv7-E6 object detector (56 FPS V100, 55.9%
AP) outperforms both transformer-based detector SWIN-
L Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by
509% in speed and 2% in accuracy, and convolutional-
based detector ConvNeXt-XL Cascade-Mask R-CNN (8.6
FPS A100, 55.2% AP) by 551% in speed and 0.7% AP
in accuracy, as well as YOLOv7 outperforms: YOLOR,
YOLOX, Scaled-YOLOv4, YOLOv5, DETR, Deformable


In [18]:
print(text_chunks[2].page_content)

DETR, DINO-5scale-R50, ViT-Adapter-B and many other
object detectors in speed and accuracy. Moreover, we train
YOLOv7 only on MS COCO dataset from scratch without
using any other datasets or pre-trained weights. Source
code is released in https://github.com/WongKinYiu/yolov7.
1. Introduction
Real-time object detection is a very important topic in
computer vision, as it is often a necessary component in
computer vision systems. For example, multi-object track-


In [20]:
print(text_chunks[151].page_content)

formers for end-to-end object detection. In Proceedings of
the International Conference on Learning Representations
(ICLR) , 2021. 10
15


In [23]:
import os
os.environ["OPENAI_API_KEY"]=""

In [24]:
embedding=OpenAIEmbeddings()

In [26]:
len(embedding.embed_query("How are you"))

1536

In [31]:
len(embedding.embed_query("hi i am fine"))

1536

In [27]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '139ffa76-6b06-47a2-8224-be3e1ae9df7c')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [28]:
import pinecone

/opt/conda/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [29]:
pinecone.init( api_key=PINECONE_API_KEY,  # find at app.pinecone.io
     environment=PINECONE_API_ENV  # next to api key in console
)

In [32]:
index_name = "testing" # put in the name of your pinecone index here

In [33]:
index = pinecone.Index('testing')


## Create Embeddings for each of the Text Chunk

In [35]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [36]:
docsearch

In [44]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x7f1fa7300df0>)

In [37]:
query= "YOLOv7 outperforms which models"

In [38]:
docs = docsearch.similarity_search(query)

In [39]:
docs

[Document(page_content='YOLOv7-tiny 6.2 3.5 320 30.8% 47.3% 32.2% 10.0% 31.9% 52.2%\nimprovement -39% -49% - = = = -0.9 = +0.7\nYOLOR-E6 [81] 115.8M 683.2G 1280 55.7% 73.2% 60.7% 40.1% 60.4% 69.2%\nYOLOv7-E6 97.2M 515.2G 1280 55.9% 73.5% 61.1% 40.6% 60.3% 70.0%\nimprovement -19% -33% - +0.2 +0.3 +0.4 +0.5 -0.1 +0.8\nYOLOR-D6 [81] 151.7M 935.6G 1280 56.1% 73.9% 61.2% 42.4% 60.5% 69.9%\nYOLOv7-D6 154.7M 806.8G 1280 56.3% 73.8% 61.4% 41.3% 60.6% 70.1%\nYOLOv7-E6E 151.7M 843.2G 1280 56.8% 74.4% 62.1% 40.8% 62.1% 70.6%'),
 Document(page_content='YOLOv5-L6 (r6.1) [23] 76.8M 445.6G 1280 63 - / 53.7% - -\nYOLOX-X [21] 99.1M 281.9G 640 58 51.5% / 51.1% - -\nYOLOv7-E6 97.2M 515.2G 1280 56 56.0% /55.9% 73.5% 61.2%\nYOLOR-E6 [81] 115.8M 683.2G 1280 45 55.8% / 55.7% 73.4% 61.1%\nPPYOLOE-X [85] 98.4M 206.6G 640 45 52.2% / 51.9% 69.9% 56.5%\nYOLOv7-D6 154.7M 806.8G 1280 44 56.6% /56.3% 74.0% 61.8%\nYOLOv5-X6 (r6.1) [23] 140.7M 839.2G 1280 38 - / 55.0% - -\nYOLOv7-E6E 151.7M 843.2G 1280 36 56.8% /56.8

In [40]:
llm = OpenAI()

In [41]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [42]:
query = "YOLOv7 outperforms which models"

In [43]:
qa.run(query)

' YOLOv7 outperforms YOLOv5-L6, YOLOX-X, YOLOR-E6, PPYOLOE-X, YOLOv5-X6, and YOLOR-CSP.'

In [45]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt:  what is a yolo?


Answer:  YOLO (You Only Look Once) is a type of object detector, specifically a deep learning algorithm used for object detection in images and videos. It was developed by Chien-Yao Wang, Alexey Bochkovskiy, and Hong-Yuan Mark Liao from the Institute of Information Science at Academia Sinica in Taiwan. YOLOv7 is the latest version of the algorithm, which has set a new state-of-the-art for real-time object detectors in terms of speed and accuracy.


Input Prompt:  who is invented the yolo?


Answer:  Joseph Redmon and Ali Farhadi


Input Prompt:  what was the accuracy of the yolov7?


Answer:  The accuracy of the YOLOv7 was 56.8% AP test-dev / 56.8% AP min-val.


Input Prompt:  exit


Exiting


SystemExit: 

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
